# 扩散映射(DiffusionMaps)

## 符号定义

|符号|概念|
|:--:|:--:|
|$\pmb{x}$|样本点|
|$X$|样本集合|
|$N$|样本总数|
|$G$|有限图|
|$S$|有限图元素集合|
|$W$|权重矩阵|
|$D$|度矩阵|
|$P$|转移矩阵|
|$M$|距离矩阵|
|$d$|距离|
|$m$|降维后维度|

## 概念
ISOMAP通过替换欧氏距离为最短路径距离实现了比较好的降维效果，但是ISOMAP有一个非常明显的缺陷：对噪声较敏感。噪声很有可能改变两个点之间的最短路径以至于影响相当多样本对的距离度量，从而得到错误的降维结果。

为了抵抗噪声的影响，一个非常简单的思路就是取多条路径的平均值（若两个点由多条短路径相连则可以断定这两个是相近的）。Diffusion Maps就有着类似的想法，不同的是，Diffusion Maps是以图的形式来审视所有数据。Diffusion Maps定义相应的距离度量用于衡量图中节点之间的连接程度，节点间相互连接的总量则作为两节点的相似程度。Difuusion Maps在随机游走以及扩散过程的基础上定义扩散距离，并找到了扩散距离与低维欧氏距离的联系，从而实现将高维空间的数据嵌入到低维空间。

Diffusion Maps主要有如下工作：

1. 定义一个有限图$G=(S, E)$，其中$S$为所有数据点构成的元素集合，$E$为所有边的集合
2. 在有限图的基础上，定义相应的转移矩阵$P$，并进行随机游走。
3. 寻找当随机游走步数$t\rightarrow \infty$时的平稳分布$\mu$
4. 定义扩散距离并实现数据的低维空间嵌入

## 推导

* **权重矩阵**

首先需要定义一个权重矩阵，权重矩阵的定义方式可以由实际情况决定，仅需要满足如下的条件：

1. 对称性
$$
\begin{equation}
    W = W^T
\end{equation}
$$
2. 元素非负
$$
\begin{equation}
    w(i, j) \geq 0
\end{equation}
$$

在下述推导中采用高斯核进行定义，即

$$
\begin{equation}
    \begin{split}
    W_{ij} 
    &= w(i, j) \\
    &= \mathcal{k}(\pmb{x_i}, \pmb{x_j}) \\
    &= \exp(-\gamma||\pmb{x_i-\pmb{x_j}}||^2)
    \end{split}
\end{equation}
$$

为了调整节点空间位置的影响，对权重矩阵进行归一化。

为了方便计算，首先引入度矩阵$D$，度矩阵为对角阵

$$
\begin{equation}
    D_{ii} = \sum_{j=1}^NW(\pmb{x_i, x_j})
\end{equation}
$$

这里采用对称归一化，即$D^{-\frac{1}{2}}WD^{-\frac{1}{2}}$，这样处理后权重矩阵依然为对称矩阵。

* **随机游走**

上述使用核函数获得的两两点之间的相似性度量实际上仅描述了局部信息。为了进一步获得全局上的信息，Diffusion Maps引入了图上的随机游走。从某一个特定的节点$\pmb{x_i}$出发，经过一个时间步后，相较于较远的节点，显然更容易转移到较近的点。因此上述由高斯核函数定义的权重矩阵能够用于构造转移矩阵。

$$
\begin{equation}
    P = D^{-1}W
\end{equation}
$$

上述仅为一个时间步的转移，Diffusion Maps引入了多个时间步的转移。设$P^t$表示t个时间步的转移矩阵，其元素$P^t_{ij}$表示经过t个时间步从节点$\pmb{x_i}$转移到节点$\pmb{x_j}$的概率。显然多个时间步的转移能够体现节点间的内在联系。实际上，当$t\rightarrow\infty$时，会得到一个平稳分布。

$$
\begin{equation}
    \mu(\pmb{x_i}) = \frac{D_{ii}}{\sum_{j=1}^ND_{jj}}
\end{equation}
$$

详细证明可以参考参考资料[3]

* **扩散距离**

在随机游走的基础上，若两个节点的t个时间步的转移矩阵相似，我们可以认为这两个节点的相似度比较高，反之则可以认为这两个节点的相似度低，Diffusion Maps正是基于此定义了一个相当有效的节点相似度度量方法。其定义如下

$$
\begin{equation}
    d^2_t(\pmb{x_i}, \pmb{x_j}) = \sum_{z=1}^N\frac{(P^t_{iz}-P^t_{jz})^2}{\mu(\pmb{x_z})}
\end{equation}
$$

上式是一个带权重的$l^2$距离，节点分布密集的区域$\mu(\pmb{x_z})$较大，此时权重较小，而节点分布稀疏的区域$\mu(\pmb{x_z})$较小，此时权重较大。

* **转移矩阵的特征分解**

这里直接给结论，详细推导见参考资料[4]

对于$P^t$有如下分解

$$
\begin{equation}
    P^t = \sum_{i=1}^N\lambda_i^t\pmb{\varphi_i}\pmb{\psi_i}
\end{equation}
$$

其中$\lambda$为特征值，$\varphi$为左特征向量，$\psi$为右特征向量。左特征向量关于$\frac{1}{\mu}$归一化，右特征向量关于$\mu$归一化，并按照特征值的大小降序排列，可以得到

$$
\begin{equation}
    P^t_{ij} = \sum_{z=1}^N\lambda_z^t\varphi_z(i)\psi_z(j)
\end{equation}
$$

其中$P^t_{ij}$为矩阵$P^t$中位置为$ij$的值，$\varphi_z(i)$为第z个左特征向量的第i个位置的值

将式9带入式7可以得到

$$
\begin{equation}
    \begin{split}
    d^2_t(\pmb{x_i}, \pmb{x_j}) 
    &= \sum_{z=1}^N\frac{(P^t_{iz}-P^t_{jz})^2}{\mu(\pmb{x_z})} \\
    &= \sum_{z=1}^N\frac{(\sum_{l=1}^N\lambda_l^t\varphi_l(i)\psi_l(z)-\sum_{l=1}^N\lambda_l^t\varphi_l(j)\psi_l(z))^2}{\mu(\pmb{x_z})} \\
    &= \sum_{z=1}^N\frac{\sum_{l=1}^N\lambda_l^{2t}\psi_l^2(z)(\varphi_l(i)-\varphi_l(j))^2}{\mu(\pmb{x_z})} \\
    &= \sum_{l=1}^N\sum_{z=1}^N\frac{\psi_l^2(z)}{\mu(\pmb{x_z})}\lambda_l^{2t}(\varphi_l(i)-\varphi_l(j))^2 \\
    &= \sum_{l=1}^N\lambda_l^{2t}(\varphi_l(i)-\varphi_l(j))^2 \\
    &= \sum_{l=1}^N(\lambda_l^{t}\varphi_l(i) - \lambda_l^{t}\varphi_l(j))^2
    \end{split}
\end{equation}
$$

考虑到$\pmb{\varphi_1} \equiv \pmb{1}$因此忽略第一项，最终得到

$$
\begin{equation}
    d^2_t(\pmb{x_i}, \pmb{x_j}) = \sum_{l=2}^N(\lambda_l^{t}\varphi_l(i) - \lambda_l^{t}\varphi_l(j))^2
\end{equation}
$$

考虑到特征值的衰减，取前几项即可达到一定的精度，因此上述定义的距离可以由前$m$项近似表示

$$
\begin{equation}
    \begin{split}
    d^2_t(\pmb{x_i}, \pmb{x_j}) 
    &= \sum_{l=2}^N(\lambda_l^{t}\varphi_l(i) - \lambda_l^{t}\varphi_l(j))^2 \\
    &\simeq \sum_{l=2}^{m+1}(\lambda_l^{t}\varphi_l(i) - \lambda_l^{t}\varphi_l(j))^2
    \end{split}
\end{equation}
$$

上述定义的距离显然可以视为在$\mathcal{R^m}$上的欧式距离。在式-12的基础上定义如下映射

$$
\begin{equation}
    \Phi_t: \pmb{x_i} \longmapsto [\lambda_2^{t}\varphi_2(i), \lambda_3^{t}\varphi_3(i), \cdots, \lambda_{m+1}^{t}\varphi_{m+1}(i)]^T
\end{equation}
$$

可以得到
$$
\begin{equation}
    \begin{split}
    d^2_t(\pmb{x_i}, \pmb{x_j}) 
    &= \sum_{l=2}^N(\lambda_l^{t}\varphi_l(i) - \lambda_l^{t}\varphi_l(j))^2 \\
    &\simeq ||\Phi_t(\pmb{x_i}) - \Phi_t(\pmb{x_j})||^2
    \end{split}
\end{equation}
$$

式-14指出在原空间定义的扩散距离可以由低维空间$\mathcal{R^m}$中的欧式距离近似表示。这一低维表示由随机游走时间$t$以及转移矩阵决定。


## 算法步骤

## 参考资料

[1] https://en.wikipedia.org/wiki/Isomap

[2] https://www.cnblogs.com/EIPsilly/p/15732378.html

[3] https://www.cs.yale.edu/homes/spielman/561/lect10-18.pdf

[4] https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-262-discrete-stochastic-processes-spring-2011/video-lectures/lecture-8-markov-eigenvalues-and-eigenvectors/MIT6_262S11_lec08.pdf